# RFSoC for SDR

*Grow this page - look at the SDR book?*

## The Software Defined Radio (SDR)

SDR simply refers to the analogue components in a system becoming software based. When software replaces hardware:

- Modulators, filters, amplifiers become DSP Software code.


- Frequency tuning is done with sine wave look-up tables


- Channel selection is done with DSP digital filters


- Sample rates are GHz, and soon to be 10's of GHz


Full RF sampling requires rates of the order of a few GHz ($10^9$ Hz). Nyquist still applies - must sample at least 2x bandwidth of the signal of interest.

*SDR diagram*

In the last few years, SDR has gone mainstream with low cost boards and support features for tools such as MATLAB to interface with the boards. Many of the current solutions are using Zynq and other boards with add-on analogue interface cards. In most cases the designs are **IF (Intermediate Frequency) samplers**.

There are three identifiable reasons for this mainstream adoption of SDRs: ADC and DAC sample rates, spectrum sharing and low cost IF radio platforms.

**SDR Driver 1: ADC and DAC Sample Rates**

Over the last 40 years, sampling rates have increased in much the same way as processing power with Moore's Law.

*Moore's Law sample rates diagram*

Current sampling rates of near 10's of GHz allow sampling of RF frequency bands. Tracking the improvement of commercially available devices since the 1960's we can see that sampling rates have increased at a rate of just more than a factor of 10 per decade.

A key driver in the 1970's was digital telephony and voiceband (8 KHz / 10 bit) devices. The early and mid-80s brought the beginning of the digital audio revolution with the CD becoming widely available with a sampling rate of 44.1 KHz (an 'odd' number derived from video DACs). By the late 80s and early 90s sigma delta technology had arrived with 'oversampling' devices running at low MHz, before decimating down or interpolating up to baseband audio.

By the start of the 1990's and the advent of GSM and 2nd Generation cellular/mobile, voiceband DACs and ADCs were widespread in mobile phones. However extensive development was now going in to IF converters, whereby sampling rates of 40 MHz and higher would digitize signals with frequency content up to half of the sampling rate. 

By the early 2000s, IF samplers operating at 100's of MHz had arrived, with devices offering multiple channels, 14 bits resolution or more. GHz sample devices became available in the mid 2000s and brought the first real SDR devices to market with the combination of Xilinx FPGA and Analog Devices ADCs/DACs.

And then the RFSoC device appeared, bringing single chip SDR to absolute reality, with on chip DACs and ADCs and an on-chip set of processors and programmable logic to implement DSP algorithms.

**SDR Driver 2: Spectrum Sharing**

The frequency spectrum is becoming more democratized and more available. For many years the spectrum has been the preserve of large MNOs (Mobile Network Operators), Government agencies and broadcasters with just a few open bands for unlicensed access (2.4 GHz, 5 GHz). This is beginning to change:

- **USA**: TV White Space (TVWS, 400 to 700 MHz), CBRS (Citizen Broadband Radio Service, 3.5 GHz)


- **UK**: In the UK in July 2019 Ofcom announced a **use it or share it** policy. Any frequency band leased/owned by an MNO must be made available for sharing if they are not using, nor intending to use it.


This creates widespread interest in private networks with disaggregated RAN, tier 1 providers and new entrants to Core and Cloud markets. Shared spectrum private mobile and wireless networks can be built by councils and new entrants to the market, with cloud and core services and connectivity from other new market entrants such as Cisco and AWS.

*could do an image here*

**SDR Driver 3: Low Cost IF Radio Platforms**

There are now many SDR boards, and software support solutions. Most of these are IF Sampling Systems - the RFSoC will be RF Sampling. 

Some example low cost platforms include:

- **Xilinx Zynq-Based Radio**: ZC706, ZedBoard, ADI FMCOMMS1/2/3/4


- **Xilinx FPGA-Based Radio**: Virtex-6 ML605, Spartan-6 SP605, ADI FMCOMMS1, Epiq FMC-1Rx


- **USRP Radio**: USRP2, N200/210, B200/B210, X300/310


- **RTL-SDR Radio**

Additionally the radios in our pockets feature many IF sampling systems and implementations. As RF sampling becomes more mainstream we can expect to see first the basestations moving to RF sampling, and at some point in the future user devices will follow. For now however, your phones are crammed full of different radios all running on different processor systems and with multiple IF samplers and ADCs and DACs. 

**Your daily 8 radios**: GSM, 3G, LTE (10s Mbits/sec), Bluetooth, 802.11 (10s Mbits/sec), FM receiver, GPS and NFC.

## RFSoC for Radio

**RFSoC Advantages**

Several of the RFSoC architecture features previously discussed give RFSoC devices an advantage over a traditional multi-chip solution for general radio designs:  

- **Very wide RF bandwidth**: The RFSoC can directly digitise a range of bands. The high sampling rates (and therefore the wide RF bandwidth) supported by the RFSoC is a significant feature. This means that all-digital radios can be designed with minimal external analogue cicuitry, for systems that cover RF bands up to roughly 4 GHz (and beyond, for the most recent RFSoC generations). This is achieved directly for RF bands up to ~2 GHz and by exploiting Nyquist zones for frequencies between ~2 GHz and ~4 GHz. The use of Nyquist zones will be explained later in this notebook.  


- **Supports different radio architectures**: either all-digital or with digital-to-IF for higher bands, to cover any desired RF frequency band. For instance, very high RF frequency bands can be catered to by integrating external modulation and demodulation to convert between IF and RF. 


- **Multiple Channels**: facilitates sophisticated, MIMO radio designs and enables a single chip to support multiple radios simultaneously. The multiple DAC and ADC channels supported by the RFSoC (8 or 16 of each) means that radio systems for MIMO applications can be developed on a single chip. There is also the potential to implement several radios for different bands and applications.


- **Single chip solution**: eases design effort, especially for interfacing, and enables systems with smaller footprint and lower power operation.


- **Integrated DUC (filtering and modulation) and DDC (demodulation and filtering)**: reduces design effort and required PL resources, and enables lower power consumption. The integrated DUC and DDC can also support higher sampling rates than would be possible on PL. 


- **Integrated FEC block**: likewise reduces design effort, PL resource consumption, and power usage. 


The integration of RF data converters with PL and hardened radio IP blocks is also an advantage for developing radio designs. DUC, DDC, and FEC functionality is commonly required, and it would otherwise need to be implemented in the PL. Particularly in the case of FEC, a considerable amount of PL resources may be needed. Having access to pre-verified and hardened blocks reduces the design effort required, and saves the PL resources that would otherwise be needed to implement them on the PL. It is also notable that the output of the DUC and the input of the DDC operate at the same sampling rates as the RF-DAC and RF-ADC respectively (up to multiple GHz), i.e. much higher rates than could be supported by a DUC and DDC implemented in the PL.

**RFSoC Key Features for SDR**

This section considers aspects of the RFSoC's functionality that are especially beneficial for SDR, i.e. beyond the needs of radio in general (which might have fixed functionality once designed). SDR requires the radio to be dynamic and flexible in its operation, under software control.

- **Programmable RF sample rate**: caters for different requirements.


- **High sample rates**: A wide RF bandwidth enables multiple radio bands to be generated/captured simultaneously. Useful for SDR where dynamic operation is required. Flexibility is further extended by the availability of multiple channels.


- **Programmable DUC and DDC parameters**: flexible frequency and decimation/interpolation ratios can be programmed from software.


- **Run-time reprogrammability**: Hardware design (on PL) can be designed to be reprogrammable at run-time, e.g. filter coefficients, operating, etc. Beyond this, the full hardware system can be reprogrammed, and Partial Reconfiguration (PR) can be exploited.


- **Single chip solution**: The integration of PS applications and real-time processors with PL, data converters and FEC block is ideal for close coupling of software control with hardware processing and RF functionality.


The RFSoC provides the means to exert dynamic control over multiple aspects of a radio's functionality. This includes the front end sampling rate and the configuration of the RF data converters, the DUC and DDC behaviour, and also custom modules that may be developed in hardware on the PL. Therefore, the modulation and demodulation frequencies can be dynamically altered, and the behaviour of hardware processing blocks can be controlled from software. Parameters of hardware processing blocks can be altered. Taking this to an extreme, software could also prompt part of or all of the PL to be reprogrammed with a new set of functionality.

The integration of the PS with the hardware elements (PL, data converters, hardened DUC, DDC, and FEC blocks) is particularly significant in this respect. The single chip architecture enables software to reside very close to the hardware that it controls, and therefore the feedback loop is much faster than would be possible with a multi-chip solution.

University Of Strathclyde &copy;